# ABS Quarterly National Accounts 5206

Expenditure approach: $GDP(E) = C + I + G + NX$

Income approach: $GDP(I) = CompensationOfEmployees + RentalAndRoyaltyIncome + BusinessCashFlow + NetInterest$

Output or product approach: $GDP(O) = (GrossValueAdded + TaxesOnProducts - SubsidiesOnProducts) - IntermediateConsumption + TaxesOnProducts - SubsidiesOnProducts$

## Python set-up

In [1]:
# system imports
import re
import textwrap
from typing import cast

# analytic imports
import numpy as np
import pandas as pd
import readabs as ra
from readabs import metacol as mc

In [2]:
# local imports
from abs_summary_plot import plot_summary
from abs_helper import get_abs_data, ANNUAL_CPI_TARGET_RANGE, QUARTERLY_CPI_TARGET
from abs_plotting import fix_abs_title
from henderson import hma
from plotting import (
    calc_growth,
    finalise_plot,
    line_plot,
    plot_covid_recovery,
    plot_growth,
    plot_growth_finalise,
    plot_revisions,
)

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False
FILE_TYPE = "png"

## Get data from ABS

In [3]:
abs_dict, meta, source, RECENT = get_abs_data("5206.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

# we use this table name quite a lot ...
KEY_AGGS = "5206001_Key_Aggregates"

In [4]:
# Let's see the spreadsheets we have captured
tw = textwrap.TextWrapper(width=80)
tw.wrap(", ".join(abs_dict.keys()))

['5206001_Key_Aggregates, 5206002_Expenditure_Volume_Measures,',
 '5206003_Expenditure_Current_Price, 5206004_Expenditure_Price_Indexes,',
 '5206005_Expenditure_Implicit_Price_Deflators, 5206006_Industry_GVA,',
 '5206007_Income_From_GDP, 5206008_Household_Final_Consumption_Expenditure,',
 '5206009_Changes_In_Inventories, 5206010_Agricultural_Income,',
 '5206011_National_Income_Account, 5206012_National_Capital_Account,',
 '5206013_NFC_Income, 5206014_PriNFC_Income, 5206015_PubNFC_Income,',
 '5206016_FC_Income, 5206017_Gen_Govt_Income_Account,',
 '5206018_Nat_Gen_Govt_Income_Account, 5206019_StateLocal_Gen_Govt_Income_Account,',
 '5206020_Household_Income, 5206021_External_Account, 5206022_Taxes,',
 '5206023_Social_Assistance_Benefits, 5206024_Selected_Analytical_Series,',
 '5206025_SFD_Summary, 5206026_SFD_NSW, 5206027_SFD_VIC, 5206028_SFD_QLD,',
 '5206029_SFD_SA, 5206030_SFD_WA, 5206031_SFD_TAS, 5206032_SFD_NT,',
 '5206033_SFD_ACT, 5206034_Key_Aggregates_and_Analytical_Series_Annual_d

In [5]:
# extract meta data, etc.
CVM = "Chain volume measures"
VALUE_TEXT = "$ Millions"

## Plot

### Summary plot

In [6]:
def summarise() -> None:
    """Summarise the data in the ABS dataset"""

    required = {
        # order from least to most important (ie. bottom to top)
        # chart label: (ABS-code, growth-over-n-periods)
        "1Q growth: Households price deflator": ("A2303940R", 1),
        "4Q growth: Households price deflator": ("A2303940R", 4),
        "1Q growth: GNE price deflator": ("A2303727C", 1),
        "4Q growth: GNE price deflator": ("A2303727C", 4),
        "Household saving ratio": ("A2323382F", 0),
        "1Q growth: GDP per Hour Worked": ("A2304192L", 1),
        "4Q growth: GDP per Hour Worked": ("A2304192L", 4),
        "1Q growth: Hours Worked": ("A2304428W", 1),
        "4Q growth: Hours Worked": ("A2304428W", 4),
        "1Q growth: Household consumption": ("A2304081W", 1),
        "4Q growth: Household consumption": ("A2304081W", 4),
        "1Q growth: GVA market sector": ("A3606056V", 1),
        "4Q growth: GVA market sector": ("A3606056V", 4),
        "1Q growth: GDP/Capita": ("A2304404C", 1),
        "4Q growth: GDP/Capita": ("A2304404C", 4),
        "1Q growth: GDP": ("A2304402X", 1),
        "4Q growth: GDP": ("A2304402X", 4),
    }

    start = "2000Q1"
    plot_summary(
        required,
        abs_dict,
        meta,
        start=start,
        title=f"Key GDP statistics {abs_dict[KEY_AGGS].index[-1]}",
        rfooter=source,
        lfooter="Australia. All values are percentages. "
        "All values are seasonally adjusted. ",
        pre_tag="summary",
        show=SHOW,
        file_type=FILE_TYPE,
    )


summarise()

Overriding x0: True


### Raw charts, COVID-recovery charts, multi-year growth charts

In [7]:
def get_wanted_items() -> list[tuple[str, str, str]]:
    """Identify the core data items we want to chart.
    Returns a list of tuples, where each tuple comprises
    the label, seasonally adjusted/trend, and table name."""

    sa, trend = "Seasonally Adjusted", "Trend"
    kags, sas = "5206001_Key_Aggregates", "5206024_Selected_Analytical_Series"

    # Let's grab all of the Seasonally Adjusted CVM data items for the Key Aggregates
    key_cvm_measures = meta.loc[
        (meta[mc.stype] == sa)
        & (
            meta[mc.did].str.contains(CVM)
            & ~(meta[mc.did].str.contains("Percentage"))
            & (meta[mc.table] == kags)
        ),
        mc.did,
    ]
    wanted = [(w, sa, kags) for w in key_cvm_measures]

    # Then we add other data items we are interested in
    wanted += [
        # [label, table, stype]
        ("GDP per hour worked: Index ;", sa, kags),  # Labour productivity
        (
            "Gross value added per hour worked market sector: Index ;",
            sa,
            kags,
        ),  # Labor productivity
        # ["Real unit of labour costs: Index ;", trend, kags),
        # ["Real unit of labour costs - Non-farm: Index ;", trend, kags),
        ("Terms of trade: Index ;", sa, kags),
        ("Public ;  Final demand: Chain volume measures ;", sa, sas),
        ("Private ;  Final demand: Chain volume measures ;", sa, sas),
        (
            "Households ;  Final consumption expenditure - Goods: Chain volume measures ;",
            sa,
            sas,
        ),
        (
            "Households ;  Final consumption expenditure - Services: Chain volume measures ;",
            sa,
            sas,
        ),
        (
            "Households ;  Final consumption expenditure - Essential consumption: Chain volume measures ;",
            sa,
            sas,
        ),
        (
            "Households ;  Final consumption expenditure - Discretionary consumption: Chain volume measures ;",
            sa,
            sas,
        ),
        ("Compensation of employees per hour: Current prices ;", sa, sas),
        ("Non-farm compensation of employees per hour: Current prices ;", sa, sas),
        (
            "Gross fixed capital formation - New private business investment: Chain volume measures ;",
            sa,
            sas,
        ),
    ]

    return wanted

In [8]:
def basic_charts(n_years=10) -> None:
    """Produce basic charts for the core data items we want to chart:
    - raw series charts
    - COVID recovery charts
    - quarterly/annual growth rates
    - multi-year compound growth rates."""

    tw = textwrap.TextWrapper(width=60)
    for label, stype, table in get_wanted_items():
        selector = {label: mc.did, stype: mc.stype, table: mc.table}
        _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
        series, units = ra.recalibrate(abs_dict[table][series_id], units)
        units = (
            f"{units} / Quarter"
            if "Index" not in units and "per hour" not in label
            else units
        )
        item_meta_data = meta.loc[meta[mc.id] == series_id].iloc[0]
        title, lfooter = fix_abs_title(label, f"Australia. {stype}. ")

        common = {
            "lfooter": lfooter,
            "rfooter": f"ABS {item_meta_data[mc.cat]} {table}",
            "show": SHOW,
            "file_type": FILE_TYPE,
            "y0": True,
        }

        # plot the raw series
        line_plot(
            series,
            dropna=True,
            starts=plot_times,
            tags=plot_tags,
            title=tw.fill(title),
            ylabel=units,
            pre_tag="raw-series-",
            width=2,
            **common,
        )

        # plot the COVID recovery
        plot_covid_recovery(
            series,
            dropna=True,
            title=tw.fill(f"{title} (COVID)"),
            ylabel=f"{units}",
            pre_tag="pre-covid-trajectory-",
            **common,
        )

        # plot the Q/A growth rates
        growth = calc_growth(series)
        for start, tag in zip(plot_times, plot_tags):
            plot_growth_finalise(
                *growth,
                from_=start,
                tag=tag,
                title=tw.fill(f"{title} growth"),
                pre_tag="growth-charts-",
                annotate=9,
                **common,  # type: ignore
            )

        # plot the multi-year compound rate annualised
        ax = plot_growth(*growth)
        if ax is None:
            continue
        multi_year_gr = (
            (series / series.shift(periods=n_years * 4) - 1) ** 1 / n_years
        ) * 100
        ax.plot(multi_year_gr.index, multi_year_gr, lw=5, c="white", label="_")
        ax.plot(
            multi_year_gr.index,
            multi_year_gr,
            lw=3,
            c="cornflowerblue",
            label=f"{n_years}-year compound annual growth rate",
        )
        finalise_plot(
            ax,
            title=tw.fill(f"{n_years}-year Growth: {title}"),
            ylabel="Per cent growth",
            legend={"loc": "best", "fontsize": 9},
            pre_tag="growth-charts-",
            **common,
        )


basic_charts()  # takes just under 20 seconds to run

### Data revisions

In [9]:
def data_revisions() -> None:
    """Plot data revisions."""

    how_far_back = 6
    dataset = [
        "Gross domestic product: Chain volume measures - Percentage changes ;",
        "GDP per capita: Chain volume measures - Percentage changes ;",
        "Gross value added market sector: Chain volume measures - Percentage changes ;",
        "Gross domestic product: Chain volume measures ;",
        "GDP per capita: Chain volume measures ;",
        "Gross value added market sector: Chain volume measures ;",
    ]
    captured = {}
    for series in dataset:
        stype = "Seasonally Adjusted"
        repository = pd.DataFrame()
        history = None
        for _i in range(how_far_back):
            # from current to historic data
            d, m = ra.read_abs_cat(
                "5206.0", single_excel_only=KEY_AGGS, history=history
            )
            selector = {series: mc.did, stype: mc.stype}
            t, s, u = ra.find_abs_id(m, selector)
            date = f"ABS print for {d[t].index[-1].strftime("%Y-%b")}"
            repository[date] = d[t][s]
            history = (d[t].index[-1] - 1).strftime("%b-%Y").lower()
        captured[series] = (repository, u)

        suffix = " Q/Q growth" if "Percentage changes" in series else ""
        plot_revisions(
            data=repository,
            units=u,
            title=f"Data revsions: {re.sub(':.*$', '', series)}{suffix}",
            rfooter=source,
            lfooter=f"Australia. {stype}. Chain volume measures. ",
            legend={"loc": "best", "fontsize": 9},
            pre_tag="data-revisions",
            y0=True,
            show=SHOW,
            file_type=FILE_TYPE,
        )

    # implied population revisions
    numerator, n_units = captured["Gross domestic product: Chain volume measures ;"]
    denominator, d_units = captured["GDP per capita: Chain volume measures ;"]
    population = numerator / denominator
    plot_revisions(
        data=population,
        units="Million Persons",
        title=f"Data revsions: Implied population from National Acounts",
        rfooter=source,
        lfooter=f"Australia. {stype}. Chain volume measures. ",
        legend={"loc": "best", "fontsize": 9},
        show=SHOW,
        file_type=FILE_TYPE,
    )
    plot_revisions(
        # implied population numerical growth
        data=population.diff(1),
        units="Million Persons",
        title=f"Data revsions: Implied quarterly population growth (Nat Acc)",
        rfooter=source,
        lfooter=f"Australia. {stype}. Chain volume measures. ",
        legend={"loc": "best", "fontsize": 9},
        show=SHOW,
        file_type=FILE_TYPE,
    )


data_revisions()

recalibrate(): Units not appropriately calibrated: Percent
recalibrate(): Units not appropriately calibrated: Percent
recalibrate(): Units not appropriately calibrated: Percent


### Long run annual GDP Growth - using a Henderson moving average

In [10]:
def chart_gdp_long_run_growth(chart: str) -> None:
    """Long-run GDP growth plots."""

    # key data sources
    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # select series
    selector = {
        table: mc.table,
        chart: mc.did,
        series_type: mc.stype,
        CVM: mc.did,
        "$": mc.unit,
    }
    _table, series_id, _units = ra.find_abs_id(meta, selector, verbose=False)

    # plot original
    ax = None  # so we can comment out a plot code-line
    q_gdp = data[series_id].dropna()
    series = (q_gdp.pct_change(periods=4) * 100).dropna()
    ax = series.plot(color="darkblue", lw=0.5, ls="-", ax=ax, label="Annual growth")

    # plot Henderson Moving Average
    henderson_n = 13  # months ==> multi-year Henderson moving average
    smoothed = hma(series, henderson_n)
    ax = smoothed.plot(
        color="darkorange",
        lw=1.5,
        ls="-",
        ax=ax,
        label=f"{henderson_n}-term Henderson moving average",
    )

    # plot decadal averages
    decades = cast(pd.PeriodIndex, series.index).year.astype(str).str[2:3].unique()
    label = "Decadal mean (DM) annual growth"
    for decade in decades:
        subset = series[
            cast(pd.PeriodIndex, series.index).year.astype(str).str[2:3] == decade
        ]
        d_series = pd.Series(
            np.repeat(subset.mean(), len(subset)), index=pd.PeriodIndex(subset.index)
        )
        ax = d_series.plot(color="darkred", lw=2, ls="--", ax=ax, label=label)
        label = "_no_legend_"
        ax.text(
            x=d_series.index[-1],
            y=series.min(),
            s=f"DM = {d_series.iloc[-1]:0.2f}%",
            rotation=90,
            ha="center",
            size="x-small",
        )

    # plot compound annual growth rate over a decade
    years = 10
    periods_per_year = 4
    annual = (
        ((q_gdp / q_gdp.shift(years * periods_per_year)) ** (1 / years)) - 1
    ) * 100
    ax = annual.plot(
        color="dodgerblue",
        lw=3,
        ls="-",
        ax=ax,
        label="Annual compound growth over decade",
    )

    # finalise the plot
    ax.legend(loc="best", prop={"size": "x-small"})
    title = f"YoY Growth - {chart} - Long-run trends"
    finalise_plot(
        ax,
        title=title,
        ylabel="Per cent / year",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()}. "
        f"{CVM}. Compared with the same quarter in the previous year.",
        pre_tag="long-run-growth-",
        y0=True,
        show=SHOW,
        file_type=FILE_TYPE,
    )

In [11]:
def do_long_run_charts():
    """Plot long-run GDP growth charts."""

    charts = [
        "Gross domestic product",
        "GDP per capita",
    ]
    for chart in charts:
        chart_gdp_long_run_growth(chart)


do_long_run_charts()

### Four quarter rolling totals - original series

In [12]:
def rolling_totals() -> None:
    """Plot 4Q rolling totals of key series."""

    table = "5206001_Key_Aggregates"
    data = abs_dict[table]
    series_type = "Original"

    charts = [
        "Gross domestic product",
        "GDP per capita",
    ]
    for chart in charts:
        selector = {
            table: mc.table,
            chart: mc.did,
            series_type: mc.stype,
            CVM: mc.did,
            "$": mc.unit,
        }
        _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
        rolling_4q = data[series_id].rolling(4).sum()
        # units = "$ Number" if units == "$" else units
        rolling_4q, units = ra.recalibrate(rolling_4q, units)
        rolling_4q.name = chart

        title = meta.loc[meta[mc.id] == series_id, mc.did].iloc[0]
        lfooter = f"Australia. 4Q rolling sum. {series_type} series. "
        title, lfooter = fix_abs_title(title, lfooter)
        plot_covid_recovery(
            rolling_4q,
            title=title,
            ylabel=f"{units} / year",
            tags="covid-annual",
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            pre_tag="4Qrolling-",
            show=SHOW,
            file_type=FILE_TYPE,
        )


rolling_totals()

### Imports and Exports (Both CVM and CP)

In [13]:
tables = {
    "Volumetric": "5206002_Expenditure_Volume_Measures",
    "Current Price": "5206003_Expenditure_Current_Price",
}
items = {
    "Exports": "Exports of goods and services ;",
    "Imports": "Imports of goods and services ;",
    "GDP": "GROSS DOMESTIC PRODUCT ;",
}
stype = "Seasonally Adjusted"
for t_label, table in tables.items():
    data = abs_dict[table]
    box = {}
    for i_label, item in items.items():
        selector = {item: mc.did, table: mc.table, stype: mc.stype}
        _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
        series = data[series_id]
        box[i_label] = series
    plotable = pd.DataFrame(
        {
            "Exports": box["Exports"] / box["GDP"] * 100,
            "Imports": box["Imports"] / box["GDP"] * 100,
        }
    )
    line_plot(
        plotable,
        title=f"Trade as a percentage of GDP ({t_label})",
        ylabel="Per cent",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {stype}. ",
        starts=plot_times,
        tags=plot_tags,
        pre_tag="trade-gdp-",
        show=SHOW,
        file_type=FILE_TYPE,
    )
    plotable = pd.DataFrame(
        {
            "Net Exports": (box["Exports"] - box["Imports"]) / box["GDP"] * 100,
        }
    )
    line_plot(
        plotable,
        title=f"Net Exports as a percentage of GDP ({t_label})",
        ylabel="Per cent",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {stype}. ",
        pre_tag="trade-gdp-",
        starts=plot_times,
        tags=plot_tags,
        show=SHOW,
        y0=True,
        file_type=FILE_TYPE,
    )



### Implicit population estimates

In [14]:
def pop_estimates() -> None:
    """Plot the implicit population from ABS National Accounts."""

    table = KEY_AGGS
    data: pd.DataFrame = abs_dict[table]
    series_type = "Original"

    pairs = {
        # $                                         $ Millions
        # (
        #    "GDP per capita: Chain volume measures ;",
        #    "Gross domestic product: Chain volume measures ;",
        # ),
        (
            "GDP per capita: Current prices ;",
            "Gross domestic product: Current prices ;",
        ),
    }

    pop_series: dict[int, pd.Series] = {}
    for i, pair in enumerate(pairs):
        data_items: dict[int, pd.Series] = {}
        for index, series in enumerate(pair):
            selector = {
                table: mc.table,
                series_type: mc.stype,
                series: mc.did,
            }
            _table, series_id, _units = ra.find_abs_id(meta, selector)
            data_items[index] = data[series_id]
        population = data_items[1] / data_items[0]
        pop_series[i] = population
    fin_pop_series = pd.DataFrame(pop_series).mean(axis="columns").dropna()
    fin_pop_series.name = "Australian population"
    print(fin_pop_series.index[0], fin_pop_series.index[-1])

    title = "Implicit population from ABS National Accounts"
    common = {
        "rfooter": f"{source} {table}",
        "lfooter": "Australia. Calculated. ",
        "pre_tag": "population-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    plot_covid_recovery(
        fin_pop_series,
        title=title,
        ylabel="Millions",
        tags="covid",
        **common,
    )

    growth = calc_growth(fin_pop_series)
    g4 = fin_pop_series.diff(periods=4) * 1000  # convert to thousands
    g1 = fin_pop_series.diff(periods=1) * 1000  # convert to thousands

    for start, tag in zip(plot_times, plot_tags):
        new_title = title.replace("population", "pop. growth rate")
        plot_growth_finalise(
            *growth,
            from_=start,
            tag=tag,
            title=new_title,
            annotate=9,
            **common,  # type: ignore
        )
        new_title = new_title.replace(" rate", "")
        plot_growth_finalise(
            g4,
            g1,
            from_=start,
            tag=tag,
            ylabel="Thousands",
            title=new_title,
            annotate=9,
            **common,
        )


pop_estimates()

1959Q3 2024Q3


### Implicit price deflators

In [15]:
def price_deflators() -> dict:
    """Calculate and plot the implicit price deflators for key series."""

    table = "5206005_Expenditure_Implicit_Price_Deflators"
    data = abs_dict[table]

    keys = {
        "GDP": "GROSS DOMESTIC PRODUCT ;",
        "GNE": "Gross national expenditure ;",
        "Households": "Households ;  Final consumption expenditure ;",
    }

    common = {
        "rfooter": f"{source}",
        "lfooter": "Australia. Seasonally adjusted. ",
        "ylabel": "Per cent growth",
        "legend": {"loc": "best", "fontsize": "xx-small"},
        "pre_tag": "deflators-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    deflators = {}  # used in calculations below ...
    for prefix, key in keys.items():
        ident = meta[(meta[mc.did] == key) & (meta[mc.table] == table)][mc.id].iloc[0]
        series = data[ident]
        deflators[prefix] = series / series.iloc[-1]  # rebase
        title = f"Growth in {prefix} Implicit Price Deflator"

        growth = calc_growth(series)
        plot_growth_finalise(
            *growth,
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            axhline=QUARTERLY_CPI_TARGET,
            title=title,
            from_=RECENT,
            tag="growth",
            annotate=9,
            zero_y=True,
            **common,  # type: ignore
        )

        growth[0].name = title
        line_plot(
            growth[0],
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            title=f"Annual {title}",
            width=2,
            y0=True,
            **common,  # type: ignore
        )

    return deflators


DEFLATORS = price_deflators()

#### Unit Labour Cost growth

In [16]:
def get_ulc() -> None:
    """Annual unit labour costs growth."""

    cat = "5206.0"
    series_ids = ["A2304402X", "A2302915V"]
    tables = [meta[meta[mc.id] == x][mc.table].iloc[0] for x in series_ids]
    series = [abs_dict[t][s] for t, s in zip(tables, series_ids)]
    ulc = series[1] / series[0]

    common = {
        "rfooter": f"{cat}: {', '.join(series_ids)}",
        "lfooter": "Australia. Seasonally adjusted. ",
        "ylabel": "Per cent growth",
        "pre_tag": "ulc-",
        "show": SHOW,
    }

    growth = calc_growth(ulc)
    plot_growth_finalise(
        growth[0],
        growth[1],
        title="Unit Labour Costs Growth",
        from_=RECENT,
        tag="growth",
        annotate=9,
        **common,
    )

    line_plot(
        growth[0],
        title="Unit Labour Costs Growth (YoY)",
        width=2,
        y0=True,
        **common,
    )


get_ulc()

### Savings - current prices

In [17]:
def excess_savings(plotable: pd.Series, common: dict, units: str) -> None:
    """Estimate excess COVID savings and subsequent disavings
    rather than use the pre-COVID trend, let's be a little
    circumspect and use $20B/Q as a reasonable threshhold."""

    # focus on the last 10 years or so
    ptable = plotable.loc[lambda x: x.index >= "2014Q4"].copy()

    thresh = 20  # $Billion / Quarter
    covid = (ptable.index >= "2020Q1") & (ptable.index <= "2022Q3")
    after = ptable.index >= "2022Q4"
    covid_savings = (ptable[covid] - thresh).sum()
    print(f"COVID Household savings: {covid_savings:.0f}$B")
    covid_disavings = (ptable[after] - thresh).sum()
    print(
        f"COVID Household disavings: {covid_disavings:.0f}$B "
        f"{covid_disavings/covid_savings:.1%}"
    )

    # plot growth
    ax = ptable.plot(lw=2, color="darkblue")
    ax.fill_between(
        ptable[covid].index,
        pd.Series([20] * len(covid[covid]), index=covid[covid]),
        ptable[covid],
        color="cornflowerblue",
    )
    ax.text(
        x=pd.Period("2021Q2", freq="Q"),  # type: ignore[arg-type]
        y=30,
        s=f"${covid_savings:.0f}B",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
    )
    ax.fill_between(
        ptable[after].index,
        pd.Series([20] * len(after[after]), index=after[after]),
        ptable[after],
        color="darkred",
    )
    ax.text(
        pd.Period("2023Q3", freq="Q"),  # type: ignore[arg-type]
        10,
        f"${covid_disavings:.0f}B",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
    )
    finalise_plot(
        ax,
        title="Excess household savings during COVID",
        ylabel=units,
        **({k: v for k, v in common.items() if k != "title"}),
    )


def savings() -> None:
    """Plot savings from the National Income Account."""

    table = "5206011_National_Income_Account"
    nia_data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    series_focus = "Net saving"

    headline_plots = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(series_focus))
    ][mc.did]

    for description in headline_plots:
        row = meta[
            (meta[mc.did] == description)
            & (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
        ].iloc[0]
        series_id, units = row[mc.id], row[mc.unit]
        units = f"{units} / Qtr"
        series = nia_data[series_id]
        series.name = "Series"
        plotable, units = ra.recalibrate(series, units)

        title = description.replace(" ;", "").replace("  ", " ").capitalize()

        common = {
            "title": title,
            "y0": True,
            "rfooter": source,
            "lfooter": f"Australia. {series_type.capitalize()} series. Current prices. ",
            "pre_tag": "saving-",
            "show": SHOW,
            "file_type": FILE_TYPE,
        }

        line_plot(
            plotable,
            starts=plot_times,
            tags=plot_tags,
            ylabel=units,
            width=2,
            **common,
        )

        plot_covid_recovery(
            series=plotable,
            tags="covid-current-prices",
            ylabel=units,
            **common,
        )

        if "Household" in title:
            excess_savings(plotable, common, units)


savings()

COVID Household savings: 275$B
COVID Household disavings: -92$B -33.4%


### Gross Value Added (GVA) by Industry

In [18]:
def gva_by_industry() -> None:
    """Plot GVA by industry."""
    # identify available series from metadata
    table = "5206006_Industry_GVA"
    data = abs_dict[table]

    series_type = "Seasonally Adjusted"
    industries = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit] == VALUE_TEXT)  # limit to money
    ]
    # industry_start = pd.Period("1974-09-01", freq="Q")

    for _index, row in industries.iterrows():
        series_id, units, industry = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units}  / Quarter"
        series, units = ra.recalibrate(data[series_id].dropna(), units)

        title = industry.replace(" ;", "").strip()
        if not title.endswith(")"):
            title = re.sub(r"^[^\(]+\(", "(", title)
        title = f" GVA: {title}"

        line_plot(
            series,
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=(f"{series_type.capitalize()}, " f"{CVM.lower()}"),
            pre_tag="gva-",
            width=2,
            show=SHOW,  # SHOW,
            file_type=FILE_TYPE,
        )


gva_by_industry()

###  Household Final Consumption Expenditure

In [19]:
def hfce() -> tuple[pd.Series, str]:
    """Get HFCE data and units - plot and return."""

    # start
    hfce_q_series, hfce_q_units = pd.Series(), ""

    # identify available series from metadata
    table = "5206008_Household_Final_Consumption_Expenditure"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    hfce_selected = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit] == "$ Millions")
        & (meta[mc.did].str.contains(CVM))  # limit to money
    ]

    for _index, row in hfce_selected.iterrows():
        series, units, description = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        plotable, units = ra.recalibrate(data[series].dropna(), units)
        plotable.name = f"{series_type.capitalize()} series"
        title = description.replace(f": {CVM} ;", "")
        separator = "\n" if len(title) > 25 else " "
        title = f"Household consumption:{separator}{title}"

        common = {
            "title": title,
            "ylabel": units,
            "rfooter": f"{source} {table}",
            "lfooter": f"Australia. {CVM.capitalize()} ",
            "pre_tag": "hfce-",
            "show": SHOW,
            "file_type": FILE_TYPE,
        }

        # plot - complete series - and recent showing covid recoverhy
        line_plot(
            plotable,
            width=2,
            **common,
        )

        plot_covid_recovery(
            plotable,
            tags="covid",
            **common,
        )

        growth = calc_growth(plotable)
        plot_growth_finalise(
            *growth,
            from_=RECENT,
            tag="growth",
            annotate=9,
            **(common | {"ylabel": "Per cent growth"}),
        )

        # remember HFCE for next cell ...
        if "FINAL CONSUMPTION EXPENDITURE" in description:
            hfce_q_series = plotable
            hfce_q_units = units

    return hfce_q_series, hfce_q_units

In [20]:
def quarterly_gdp() -> tuple[pd.Series, str]:
    """Get GDP series and units."""

    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    item = "Gross domestic product"

    selector = {
        table: mc.table,
        item: mc.did,
        series_type: mc.stype,
        CVM: mc.did,
        "$": mc.unit,
    }
    _table, sid, units = ra.find_abs_id(meta, selector, verbose=False)
    series = data[sid]
    return ra.recalibrate(series, units)

In [21]:
def hfce_percent_gdp():
    """Plot HFCE as a proportion of GDP."""

    hfce_q_series, _hfce_q_units = hfce()
    gdp_q_series, _gdp_q_units = quarterly_gdp()

    hfce_proportion = hfce_q_series / gdp_q_series * 100
    line_plot(
        hfce_proportion,
        title="HFCE as a proportion of GDP",
        ylabel="Per cent",
        rfooter=f"{source} 1, 8",
        lfooter=(f"Seasonally adjusted series. {CVM.lower()}"),
        pre_tag="hfce-",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
    )
    print(hfce_proportion.mean())


hfce_percent_gdp()

49.44802977623637


### Taxes

In [22]:
def taxes():
    """Plot taxes from the National Income Account."""

    # identify available series from metadata
    table = "5206022_Taxes"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    tax_plots = meta[(meta[mc.table] == table) & (meta[mc.stype] == series_type)]

    for _index, row in tax_plots.iterrows():
        # data capture
        ident, units, desc = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        series, units = ra.recalibrate(data[ident], units)

        # plot
        title = desc.replace(" ;", "")
        # a longer run on taxes is appropriate 24Q = 6 years
        start = (series.index[-1] - 24).strftime("%Y-%m-%d")
        line_plot(
            series,
            starts=start,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. Current Prices.",
            pre_tag="taxes-",
            width=2,
            show=SHOW,
            file_type=FILE_TYPE,
        )


taxes()

### Household income

In [23]:
def disposable_income():
    """Plot disposable income as a proportion of total income."""

    # Household gross disposable income = The amount of income
    # that households have available for spending after deducting
    # any taxes paid, interest payments and transfers overseas.

    table = (
        "5206020_Household_Income"  # Table 20. Household Income Account, Current prices
    )
    data = abs_dict[table]

    series_type = "Seasonally Adjusted"
    price_type = "Current prices"
    items = "GROSS DISPOSABLE INCOME", "TOTAL GROSS INCOME"
    series = []
    for item in items:
        series_id = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did].str.contains(item))
            & (meta[mc.tdesc].str.contains(price_type))
        ][mc.id].iloc[0]
        series.append(data[series_id])
    percent_di = series[0] / series[1] * 100

    line_plot(
        percent_di,
        starts=(None, RECENT),
        title=f'Households: {" / ".join(items).title()}',
        ylabel="Per cent",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{price_type.capitalize()}. ",
        pre_tag="household_income-",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
    )


disposable_income()

In [24]:
def tax_percent_income():
    """Plot Gross Individual Income Tax as a proportion of Gross Household Income."""

    # Common
    series_type = "Seasonally Adjusted"
    price_type = "Current prices"

    # Total Gross Household Income
    table = (
        "5206020_Household_Income"  # Table 20. Household Income Account, Current prices
    )
    tables = [table]
    data = abs_dict[table]
    data_desc = "TOTAL GROSS INCOME"

    tghi_id = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(data_desc))
        & (meta[mc.tdesc].str.contains(price_type))
    ][mc.id].iloc[0]
    tghi = data[tghi_id]

    # Total Income Taxes on Individuals
    table = "5206022_Taxes"  # Table 22. Taxes, Current prices
    tables.append(table)
    data = abs_dict[table]
    data_desc = "Taxes on income - Individuals - Total"
    titi_id = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(data_desc))
        & (meta[mc.tdesc].str.contains(price_type))
    ][mc.id].iloc[0]
    titi = data[titi_id]

    # Percent tax
    percent_tax = titi / tghi * 100
    line_plot(
        percent_tax,
        starts=(None, RECENT),
        title="Gross Individual Income Tax / Gross Household Income",
        ylabel="Per cent",
        rfooter=f"{source} {', '.join(tables)}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{price_type.capitalize()}. ",
        pre_tag="taxes-",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
    )


tax_percent_income()

### Government Benefits Payments

In [25]:
def benefits():
    """Plot selected social assistance benefits."""
    table = "5206023_Social_Assistance_Benefits"
    data = abs_dict[table]
    series_type = "Original"

    payments = meta[(meta[mc.table] == table) & (meta[mc.stype] == series_type)]

    for _index, row in payments.iterrows():
        ident, units, plot = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        series = data[ident].dropna()
        series, units = ra.recalibrate(series, units)

        # plot
        title = plot.replace("General government - National ;  ", "Federal Govt - ")
        title = title.replace(
            "General government - State and local ;  ", "State or Local Govt - "
        )
        title = title.replace("General government ;  ", "All Govt - ")
        title = title.replace(" ;", "")
        start = (series.index[-1] - 24).strftime("%Y-%m-%d")  # 6 years
        line_plot(
            series,
            starts=start,
            title=title,
            pre_tag="payments-",
            ylabel=f"{units}",
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. Current prices.",
            width=2,
            show=SHOW,
            file_type=FILE_TYPE,
        )


benefits()

### Total Wages v Total Profit

In [26]:
def plot_profits():
    """Plot profits as a share of profits plus wages."""
    wages_table = "5206007_Income_From_GDP"
    series_type = "Seasonally Adjusted"  # 'Original'
    _table, wages_id, wage_units = ra.find_abs_id(
        meta,
        {
            wages_table: mc.table,
            series_type: mc.stype,
            "Compensation of employees ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    wages = abs_dict[wages_table][wages_id]

    income_table = "5206007_Income_From_GDP"
    _table, tf_income_id, tf_income_units = ra.find_abs_id(
        meta,
        {
            income_table: mc.table,
            series_type: mc.stype,
            "Total factor income ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    tf_income = abs_dict[income_table][tf_income_id]

    if wage_units != tf_income_units:
        raise ValueError

    profit_share_table = "5206024_Selected_Analytical_Series"
    _table, profit_ratio_id, _share_units = ra.find_abs_id(
        meta,
        {
            profit_share_table: mc.table,
            "Seasonally Adjusted": mc.stype,
            "Profits share of total factor income: Ratio ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    profit_tfi_share = abs_dict[profit_share_table][profit_ratio_id] / 100
    profits = profit_tfi_share * tf_income

    # plot shares
    profit_wage_share = profits / (profits + wages) * 100
    hma_term = 7
    trend = hma(profit_wage_share, hma_term)
    df = pd.DataFrame(
        {
            "Profit share": profit_wage_share,
            "Henderson moving avergae": trend,
        }
    )
    line_plot(
        df,
        width=[1, 3],
        title="Profits as a share of profits plus wages",
        ylabel="Per cent",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{hma_term}-term Henderson moving average. ",
        pre_tag="profits-",
        show=SHOW,
        file_type=FILE_TYPE,
    )

    # plot values
    assert "Millions" in wage_units
    df = pd.DataFrame(
        {
            "Wages": wages / 1000,
            "Profits": profits / 1000,
        }
    )
    line_plot(
        df,
        title="Profits vs Wages",
        ylabel="$ Billions",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        pre_tag="profits-",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
    )

    # plot index
    assert wages.index[0] == profits.index[0]
    df = pd.DataFrame(
        {
            "Wages": wages / wages.iloc[0],
            "Profits": profits / profits.iloc[0],
        }
    )
    line_plot(
        df,
        title="Profits index vs Wages index",
        ylabel=f"Index ({wages.index[0]} = 1)",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        pre_tag="profits-",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_profits()

### Real Wages (using implicit price deflator)

In [27]:
def real_wages(deflators: dict[str, pd.Series]) -> None:
    """Plot real wages."""

    table = "5206024_Selected_Analytical_Series"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    price_type = "Current Prices"
    dids = [
        "Average compensation per employee: Current prices ;",
        "Compensation of employees per hour: Current prices ;",
    ]

    for did in dids:
        deflator = "Households"
        row = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did] == did)
        ].iloc[0]
        series_id, units, did = row[mc.id], row[mc.unit], row[mc.did]
        series = (data[series_id] / deflators[deflator]).dropna()
        title = did.split(":")[0].strip()
        series.name = title

        suffix = "" if "per hour" in did else " / Qtr"
        common = {
            "title": f"Real {title}",
            "ylabel": f"{units} (inflation adjusted){suffix}",
            "rfooter": f"{source} {table}",
            "lfooter": f"Australia. {series_type.capitalize()} series. "
            f"{price_type.capitalize()} adjusted by {deflator} deflator. ",
            "pre_tag": "wages-",
            "show": SHOW,
            "file_type": FILE_TYPE,
        }

        line_plot(
            series,
            width=2,
            **common,
        )

        plot_covid_recovery(
            series,
            tags="covid",
            **common,
        )


real_wages(DEFLATORS)

## Government consumption

Government final consumption expenditure (GFCE) is current expenditure by general government bodies on services to the community such as defence, education, and public order and safety. Because these are provided free of charge or at charges which cover only a small proportion of costs, the government is considered to be the consumer of its own output. Government output has no directly observable market value, and so it is valued in the national accounts at its cost of production. 

In [28]:
if False:
    table = "5206002_Expenditure_Volume_Measures"
    print(tw.fill(", ".join(meta[meta[mc.table] == table][mc.did])))

In [29]:
def govt_consumption() -> None:

    table = "5206002_Expenditure_Volume_Measures"
    data = abs_dict[table]
    stype = "Seasonally Adjusted"
    unit = "$ Millions"

    selector = {
        table: mc.table,
        stype: mc.stype,
        "GROSS DOMESTIC PRODUCT": mc.did,
        unit: mc.unit,
    }
    _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
    gdp = data[series_id]

    wanted = (
        "General government ;  Final consumption expenditure ;",
        "General government - National ;  Final consumption expenditure - Defence ;",
        "General government - National ;  Final consumption expenditure - Non-defence ;",
        "General government - National ;  Final consumption expenditure ;",
        "General government - State and local ;  Final consumption expenditure ;",
    )
    for w in wanted:
        selector = {
            table: mc.table,
            stype: mc.stype,
            w: mc.did,
            unit: mc.unit,
        }
        _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
        series = data[series_id]
        adj_series, units = ra.recalibrate(series, units)
        series.name = w
        line_plot(
            adj_series,
            title=w,
            ylabel=units,
            rfooter=f"{source}",
            lfooter=f"Australia. {stype.capitalize()} series. {CVM.capitalize()}. ",
            pre_tag="govt-consumption-",
            width=2,
            file_type=FILE_TYPE,
            show=SHOW,
        )

        tw = textwrap.TextWrapper(width=60)
        percent_gdp = series / gdp * 100
        line_plot(
            percent_gdp,
            title=tw.fill(f"{w} as a proportion of GDP"),
            ylabel="Per cent",
            rfooter=f"{source}",
            lfooter=f"Australia. {stype.capitalize()} series. {CVM.capitalize()}. ",
            pre_tag="govt-consumption-",
            width=2,
            file_type=FILE_TYPE,
            show=SHOW,
        )


govt_consumption()

In [30]:
# watermark
%load_ext watermark
%watermark --python --machine --iversions --watermark

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

numpy  : 1.26.4
re     : 2.2.1
readabs: 0.0.20
pandas : 2.2.3

Watermark: 2.5.0



In [31]:
print("Finished")

Finished
